<a href="https://colab.research.google.com/github/mochangil/K-water_Drought/blob/dev/K_water_DroughtInfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2007~2020까지의 가뭄 발생에 대한 피해내역
공공데이터포털의 가뭄피해내역 csv파일 이용 (raw data)

피해종료일이 0인 것과 피해인구가 0인 자료들은 이상치로 여겨 제외했습니다.

##사용한 데이터
*한국수자원공사가뭄피해내역(07~20)*
*한국수자원공사*


##output 데이터
1. 년도별 구분 가뭄피해 발생내역
2. 역대 가뭄피해 발생내역


#드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Drought_df = pd.read_csv('/content/drive/MyDrive/K-water/한국수자원공사가뭄피해내역.csv',encoding='cp949')

In [ ]:
Drought_df.columns

Index(['시도', '시군구', '읍면동', '피해유형', '피해인구', '피해시작일', '피해종료일'], dtype='object')

#데이터 전처리

In [ ]:
#데이터 전처리 - 피해시작일, 피해종료일의 str to datetime, 피해지속기간 column 추가,
# 피해종료일 누락 제외, 피해인구 0 제외
def data_preprocessing():
  Drought_df['피해시작일']=pd.to_datetime(Drought_df['피해시작일'])
  Drought_df['피해종료일']=pd.to_datetime(Drought_df['피해종료일'])
  Drought_df['피해지속기간'] = Drought_df['피해종료일']-Drought_df['피해시작일']
for i in range(len(Drought_df['피해인구'])):
    if Drought_df['피해인구'][i] == 0:
      print(i)
      Drought_df.drop(i)
      i -= 1

data_preprocessing()
Drought_df

,시도,시군구,읍면동,피해유형,피해인구,피해시작일,피해종료일,피해지속기간
0,전라남도,완도군,청산면,제한급수,1200,2007-10-15,2008-04-21,189 days
1,전라남도,완도군,보길면,제한급수,1800,2007-12-10,2008-05-28,170 days
2,전라남도,완도군,노화읍,제한급수,1500,2007-12-10,2008-05-28,170 days
3,경상북도,의성군,사곡면,운반급수,88,2008-01-08,2008-04-08,91 days
4,전라남도,완도군,소안면,제한급수,2458,2008-02-12,2008-05-28,106 days
...,...,...,...,...,...,...,...,...
1549,강원도,태백시,삼수동,운반급수,2,2020-12-25,2020-12-27,2 days
1550,강원도,정선군,정선읍,운반급수,26,2020-12-27,2021-02-28,63 days
1551,강원도,춘천시,사북면,운반급수,84,2020-12-30,2021-03-10,70 days
1552,강원도,정선군,여량면,운반급수,50,2020-12-30,2021-02-25,57 days


In [ ]:
Drought_df
Drought_df['years']=Drought_df['피해시작일'].dt.year

#구현할 api들
0. 년도별, 시도별, 시군구별, 읍면동별 검색
1. 년도별 발생지역 리스트
2. 년도별, 지역별 피해지속기간 리스트
3. (년도통합) 지역별 피해총인구, 피해총기간, 발생건수

#0 to 2 : 년도별 구분

In [ ]:
#1. 년도별 발생지역 리스트

# loc 변수를 시도, 시군구, 읍면동 바꿈에 따라 나오는 리스트가 바뀐다.

# 검색하고싶은 년도 입력
year = 2018

def location_per_year(currentyear):
  is_exist =[]
  # loc=Drought_df.loc[(Drought_df['years']==currentyear)]['시도']
  loc=Drought_df.loc[(Drought_df['years']==currentyear)]['시군구']
  #loc=Drought_df.loc[(Drought_df['years']==currentyear)]['읍면동']
  for i in loc:
    if i not in is_exist :
      is_exist.append(i)
    else:
      continue
  return is_exist

# print(location_per_year(year))


In [ ]:
datas = pd.DataFrame(columns=['년도','지역', '총인구(명)', '횟수', '총기간(일)','최대피해일수'])
datas.loc[0]=['년도','지역', '총인구(명)', '횟수', '총기간(일)','최대피해일수']
# print(datas)

   년도  지역  총인구(명)  횟수  총기간(일)  최대피해일수
0  년도  지역  총인구(명)  횟수  총기간(일)  최대피해일수


In [ ]:
#2. 년도별, 지역별 피해지속기간 리스트

# 검색할 년도 입력

count = 1

for year in range(2007,2021):
  list1 = location_per_year(year)
  for i in list1:
    days_list = []
    pop_list = []
    location_df = Drought_df.loc[(Drought_df['시군구']==i)]
    highloc_list = location_df.loc[(location_df['years']==year)]['시도']
    days_list = location_df.loc[(location_df['years']==year)]['피해지속기간'].dt.days
    pop_list = location_df.loc[(location_df['years']==year)]['피해인구']

    length = len(days_list)+1
    total = sum(days_list)
    longest = max(days_list)
    shortest = min(days_list)
    pop = max(pop_list)

    print(length,total,longest,shortest,pop)
    totalpop = max(pop_list)
    j = 0
    # for j in range(len(days_list)):
    #   totalpop += days_list[j] * pop_list[j]
    # datas.loc[count]=[year,i,pop,length,total]
    count += 1

print(datas)
  # print(i)
  # print("\n총",length,"개 지역\n","최장가뭄지속기간",longest,"\n","최단 가뭄지속기간",shortest,"\n")
  # print("\n총",total,"일동안 지속\n")
  # print("평균",int(total/length),"일동안 지속\n")
  # print("next\n\n\n\n\n\n")

#3: 지역별 년도통합 자료



In [ ]:
Drought_df['지역명']=Drought_df['시도']+" "+Drought_df['시군구']

In [ ]:
is_exist=[]
for i in range(len(Drought_df['지역명'])):
  if Drought_df['지역명'][i] in is_exist:
    continue
  else:
    is_exist.append(Drought_df['지역명'][i])

In [ ]:
datas = pd.DataFrame(columns=['지역', '총 피해인구(명)', '총 발생건수', '총기간(일)','최대피해일수(건)','최대피해인구(건)'])
datas.loc[0]=['지역', '총 피해인구(명)', '총 발생건수', '총기간(일)','최대피해일수(건)','최대피해인구(건)']
print(datas)

   지역  총 피해인구(명)  총 발생건수  총기간(일)  최대피해일수(건)  최대피해인구(건)
0  지역  총 피해인구(명)  총 발생건수  총기간(일)  최대피해일수(건)  최대피해인구(건)


In [ ]:
count = 0
for i in is_exist:
  count += 1
  population = Drought_df.loc[(Drought_df['지역명']==i)]['피해인구']
  day = Drought_df.loc[(Drought_df['지역명']==i)]['피해지속기간']
  totpop = sum(population)
  maxpop = max(population)
  cases = len(day)

  totday = sum(day.dt.days) + cases
  maxday = max(day.dt.days) + 1
  # print("인구",population,"\n날짜",days,"\n건수",len(days))
  print(totpop, totday,len(population),len(day),maxpop,maxday)
  datas.loc[count]=[i,totpop,len(day),totday,maxday,maxpop]

60459 5252 35 35 4978 364
6847 419 40 40 4018 92
3225 2286 20 20 1000 366
2249 397 27 27 1300 263
523 303 6 6 158 80
55577 857 26 26 16259 79
6675 1229 47 47 4800 120
1264 581 41 41 110 33
799 254 4 4 381 64
772 56 4 4 379 26
3127 353 48 48 199 36
2948 3811 50 50 185 335
6841 1390 60 60 700 367
3938 414 56 56 500 107
2770 372 28 28 400 140
7403 594 29 29 2444 106
367 36 4 4 243 31
2660 476 54 54 500 119
495 321 10 10 165 111
568 8 8 8 198 1
2442 264 12 12 411 39
1587 994 29 29 160 291
422 10 10 10 79 1
8517 1393 141 141 759 173
1181 174 16 16 160 30
362 314 12 12 97 37
2744 1336 40 40 191 282
2472 323 9 9 806 114
976 662 11 11 163 138
2856 476 24 24 1072 68
733 24 4 4 342 11
717 653 8 8 169 93
386 206 2 2 193 124
23328 307 6 6 11631 79
8400 7508 109 109 200 368
830 179 9 9 200 109
955 973 8 8 150 261
1347 510 20 20 183 89
4220 2219 33 33 275 280
100 9 1 1 100 9
57 46 3 3 21 25
374 118 8 8 66 36
61 49 3 3 35 26
916 478 29 29 136 46
216 909 4 4 93 273
14560 4093 53 53 2122 343
219 239 5 

In [ ]:
print(datas)

          지역  총 피해인구(명)  총 발생건수  총기간(일)  최대피해일수(건)  최대피해인구(건)
0         지역  총 피해인구(명)  총 발생건수  총기간(일)  최대피해일수(건)  최대피해인구(건)
1   전라남도 완도군      60459      35    5252        364       4978
2   경상북도 의성군       6847      40     419         92       4018
3   전라남도 고흥군       3225      20    2286        366       1000
4    강원도 홍천군       2249      27     397        263       1300
..       ...        ...     ...     ...        ...        ...
93  경상남도 밀양시         24       1       1          1         24
94  전라북도 남원시       1131       7       7          1        357
95  경상남도 함양군       1831       2       2          1       1783
96  경상북도 문경시         20       1       1          1         20
97  경상남도 거제시         90       1     381        381         90

[98 rows x 6 columns]


# 데이터 점수화

In [ ]:
#if necessary
#datas = datas.drop(0)

KeyError: ignored

In [ ]:
maxcases = max(datas['총 발생건수'])
maxpopulation = max(datas['총 피해인구(명)'])
maxdays = max(datas['총기간(일)'])

In [ ]:
datas['발생건수-백분위'] = datas['총 발생건수']/maxcases
datas['피해인구-백분위'] = datas['총 피해인구(명)']/maxpopulation
datas['총기간-백분위'] = datas['총기간(일)']/maxdays


In [ ]:
datas = datas.round({'발생건수-백분위':4,'피해인구-백분위':4,'총기간-백분위':4})

In [ ]:
datas['rank-발생건수'] = datas['발생건수-백분위'].rank(ascending=False)
datas['rank-피해인구'] = datas['피해인구-백분위'].rank(ascending=False)
datas['rank-총기간'] = datas['총기간-백분위'].rank(ascending=False)


In [ ]:
datas['points'] = datas['발생건수-백분위']*10 + datas['피해인구-백분위']*10 + datas['총기간-백분위']*10

In [ ]:
datas['RANK'] = datas['points'].rank(ascending=False)

In [ ]:
datas

,지역,총 피해인구(명),총 발생건수,총기간(일),최대피해일수(건),최대피해인구(건),발생건수-백분위,피해인구-백분위,총기간-백분위,rank-발생건수,rank-피해인구,rank-총기간,points,RANK,시도,시군구
1,전라남도 완도군,60459,35,5252,364,4978,0.248227,0.366832,0.699521,15.0,2.0,2.0,13.145792,2.0,전라남도,완도군
2,경상북도 의성군,6847,40,419,92,4018,0.283688,0.041544,0.055807,13.5,12.0,27.0,3.810389,19.0,경상북도,의성군
3,전라남도 고흥군,3225,20,2286,366,1000,0.141844,0.019568,0.304475,24.5,18.0,5.0,4.658867,15.0,전라남도,고흥군
4,강원도 홍천군,2249,27,397,263,1300,0.191489,0.013646,0.052877,21.0,29.0,29.0,2.58012,25.0,강원도,홍천군
5,경상남도 의령군,523,6,303,80,158,0.042553,0.003173,0.040357,54.0,53.0,39.0,0.860834,48.0,경상남도,의령군
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,경상남도 밀양시,24,1,1,1,24,0.007092,0.000146,0.000133,92.5,94.0,96.0,0.07371,96.0,경상남도,밀양시
94,전라북도 남원시,1131,7,7,1,357,0.049645,0.006862,0.000932,50.5,37.0,85.5,0.5744,59.0,전라북도,남원시
95,경상남도 함양군,1831,2,2,1,1783,0.014184,0.011109,0.000266,82.5,30.0,92.5,0.255603,86.0,경상남도,함양군
96,경상북도 문경시,20,1,1,1,20,0.007092,0.000121,0.000133,92.5,96.0,96.0,0.073467,97.0,경상북도,문경시


In [ ]:
datas['시도'] = datas.지역.str.split(' ').str[0]
datas['시군구'] = datas.지역.str.split(' ').str[1]

In [ ]:
datas

,지역,총 피해인구(명),총 발생건수,총기간(일),최대피해일수(건),최대피해인구(건),발생건수-백분위,피해인구-백분위,총기간-백분위,rank-발생건수,rank-피해인구,rank-총기간,points,RANK,시도,시군구
1,전라남도 완도군,60459,35,5252,364,4978,0.248227,0.366832,0.699521,15.0,2.0,2.0,13.145792,2.0,전라남도,완도군
2,경상북도 의성군,6847,40,419,92,4018,0.283688,0.041544,0.055807,13.5,12.0,27.0,3.810389,19.0,경상북도,의성군
3,전라남도 고흥군,3225,20,2286,366,1000,0.141844,0.019568,0.304475,24.5,18.0,5.0,4.658867,15.0,전라남도,고흥군
4,강원도 홍천군,2249,27,397,263,1300,0.191489,0.013646,0.052877,21.0,29.0,29.0,2.58012,25.0,강원도,홍천군
5,경상남도 의령군,523,6,303,80,158,0.042553,0.003173,0.040357,54.0,53.0,39.0,0.860834,48.0,경상남도,의령군
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,경상남도 밀양시,24,1,1,1,24,0.007092,0.000146,0.000133,92.5,94.0,96.0,0.07371,96.0,경상남도,밀양시
94,전라북도 남원시,1131,7,7,1,357,0.049645,0.006862,0.000932,50.5,37.0,85.5,0.5744,59.0,전라북도,남원시
95,경상남도 함양군,1831,2,2,1,1783,0.014184,0.011109,0.000266,82.5,30.0,92.5,0.255603,86.0,경상남도,함양군
96,경상북도 문경시,20,1,1,1,20,0.007092,0.000121,0.000133,92.5,96.0,96.0,0.073467,97.0,경상북도,문경시


In [ ]:
datas2 = datas.sort_values(by=['총 발생건수'],axis=0,ascending=False)
datas3 = datas.sort_values(by=['총 피해인구(명)'],axis=0,ascending=False)

In [ ]:
datas2

,지역,총 피해인구(명),총 발생건수,총기간(일),최대피해일수(건),최대피해인구(건),발생건수-백분위,피해인구-백분위,총기간-백분위,rank-발생건수,rank-피해인구,rank-총기간,points,RANK,시도,시군구
24,충청북도 충주시,8517,141,1393,173,759,1.0,0.051676,0.185535,1.0,9.0,8.0,12.372119,3.0,충청북도,충주시
35,경기도 가평군,8400,109,7508,368,200,0.77305,0.050967,1.0,2.0,10.0,1.0,18.240162,1.0,경기도,가평군
13,강원도 화천군,6841,60,1390,367,700,0.425532,0.041507,0.185136,3.0,13.0,9.0,6.521752,8.0,강원도,화천군
14,강원도 원주시,3938,56,414,107,500,0.397163,0.023894,0.055141,4.0,16.0,28.0,4.761979,14.0,강원도,원주시
18,경상북도 안동시,2660,54,476,119,500,0.382979,0.016139,0.063399,5.0,25.0,24.5,4.625172,16.0,경상북도,안동시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,충청남도 태안군,10,1,101,101,10,0.007092,0.000061,0.013452,92.5,97.0,58.0,0.206052,87.0,충청남도,태안군
93,경상남도 밀양시,24,1,1,1,24,0.007092,0.000146,0.000133,92.5,94.0,96.0,0.07371,96.0,경상남도,밀양시
63,경상북도 포항시남구,44,1,61,61,44,0.007092,0.000267,0.008125,92.5,92.0,62.0,0.154838,89.0,경상북도,포항시남구
96,경상북도 문경시,20,1,1,1,20,0.007092,0.000121,0.000133,92.5,96.0,96.0,0.073467,97.0,경상북도,문경시


# 데이터 자료화

In [ ]:
# datas.to_csv('/content/drive/MyDrive/K-water/datas.csv',header=False,index=False,encoding='utf-8-sig')
# datas2.to_csv('/content/drive/MyDrive/K-water/TotalCasesdata.csv',header=False,index=False,encoding='utf-8-sig')
# datas3.to_csv('/content/drive/MyDrive/K-water/TotalPopulationdata.csv',header=False,index=False,encoding='utf-8-sig')

#비상급수 지역구분
tableau 지오코딩을 위한 구분


In [ ]:
dataf = pd.read_excel('/content/drive/MyDrive/K-water/waterfinal.xlsx')

In [ ]:
dataf

In [ ]:
dataf['시군구'] = dataf.지역.str.split(' ').str[1]
dataf['시도'] = dataf.지역.str.split(' ').str[0]

In [ ]:
dataf

In [ ]:
dataf.to_excel('/content/drive/MyDrive/K-water/waterfinal.xlsx',header=False,index=False)